# Mineração de Dados Massivos
## Analise de descrição de mercadorias de Notas Fiscais Eletrônicas

# Ambiente

## Bibliotecas

In [1]:
# Iniciando Ambiente
import numpy as np, pandas as pd, time, random
import datetime as dt
import re 


from nltk.corpus import stopwords
import nltk
import unicodedata

# Se necessário, descomente as linhas a seguir para ler as stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Módulos

In [2]:
# Módulos de código de programação
#!pip install import-ipynb
import import_ipynb
from carga import *
from tratamento import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


importing Jupyter notebook from carga.ipynb
importing Jupyter notebook from tratamento.ipynb


<string>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 

In [ ]:
# Contador de tempo
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Duração: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

## Variáveis de Ambiente

In [ ]:
# Pasta aonde estão os dados
diretorio = '..\\dados\\'
#diretorio = "F:\\Weisner\\Documentos\\MEGA Estudos\\UNB - MDM - Mineração de Dados Massivos\\Artigo\\" 


## Abre arquivo de dados

In [ ]:
# Carrega os dados
df_original = doCarga(diretorio, opcao=2)
print ("{} registros carregados".format(df_original.size))

10153810 registros carregados


In [25]:
## SANDBOX
tic()
df_original = doTratamento(df_original)
df_original.NovaDescricao
tac()

Time passed: 0hour:0min:1sec


# Prepração dos Dados

In [26]:
%matplotlib inline
df_original['Categoria'].value_counts().plot.bar()

KeyError: 'Categoria'

In [ ]:
from sklearn.model_selection import train_test_split
##from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import sklearn

In [ ]:
# Função para avaliação dos modelos

def avalia_modelo(clf, X, y):
    resultados = sklearn.model_selection.cross_val_predict(clf, X, y, cv=5)
    # print( pd.crosstab(y, resultados, rownames=['Real'], colnames=['Predito'], margins=True))
    return np.mean(sklearn.model_selection.cross_val_score(clf, X, y, cv=5))

In [ ]:
# Obtem lista de fornecedores com mais de 10000 itens
fornecedores = df['Remetente'].value_counts()
mask = fornecedores > 10000
fornecedores = list(fornecedores[mask].index)

In [ ]:
len( fornecedores )

In [ ]:
df_resultado     = [ [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] ]
for i in range( len( fornecedores ) ):
    df_resultado.append( [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] )


In [ ]:
fornecedores

In [ ]:
df = df_original.copy()

In [ ]:
# cria o df de resultados
colunas = [ 'Remetente','registros1','precisao1','registros2','precisao2','registros3','precisao3','registros4','precisao4' ]
#df_resultado = [] #pd.DataFrame(columns = colunas)

for fonteDados in range(0,2):
    # fonteDados = 0 --> dados com as palavras separadas manualmente
    # fonteDados = 1 --> dados sem as palavras separadas manualmente
    
    linha  = 0
    coluna = 0

    # retira as palavras separadas manualmente
#    if fonteDados == 10:
#        print( )
#        print('Sem palavras selecionadas manualemnte.')
#        print()
#        
#        # Lê o arquivo de paravras indesejadas 
#        df_palavras_a_eliminar = pd.read_csv('ETL_base_lista_palavras_a_retirar_minusculas.csv', sep=";")
#        from nltk.corpus import stopwords
#        stopwords = set( stopwords.words('portuguese') + list( df_palavras_a_eliminar['palavra'] ) )
#
#        # retira as palavras indesejadas do df
#        novadescricao = map( limpaTexto2, df['NovaDescricao'] ) 
#        df.insert( loc = 2, column = "NovaDescricao2", value = list(novadescricao))#, allow_duplicates = False)
#        del novadescricao 
#        del df['NovaDescricao']
#        df.columns = ['Remetente', 'NovaDescricao', 'Categoria']  
        
#        linha  = 0
#        coluna = 4
        
    # Cria um Pipeline - Classificador Composto
    # vectorizer => transformer => classifier 
    text_clf = Pipeline([('counts' , CountVectorizer() ),
                       #  ('tfidf', TfidfTransformer()),
                         ('classifier'  , MultinomialNB()   ),
                        ])
    
    # registra resultados
    df_resultado[linha][          0] = 'Todos'
    df_resultado[linha][ coluna + 1] = df.shape[0]
    df_resultado[linha][ coluna + 2] = avalia_modelo(text_clf, df['NovaDescricao'], df['Categoria'])
    
    print(linha,' : ',coluna+1)
    print( df_resultado[linha][          0] )
    print( df_resultado[linha][ coluna + 1] )
    print( df_resultado[linha][ coluna + 2] )
    print()
    
    # Cria um Pipeline - Classificador Composto
    # vectorizer => transformer => classifier 
    text_clf = Pipeline([('counts' , CountVectorizer() ),
                         ('tfidf', TfidfTransformer()),
                         ('classifier'  , MultinomialNB()   ),
                        ])
    
    # registra resultados
    df_resultado[linha][ coluna + 3] = df.shape[0]
    df_resultado[linha][ coluna + 4] = avalia_modelo(text_clf, df['NovaDescricao'], df['Categoria'])    
    
    print(linha,' : ',coluna+3)
    print( df_resultado[linha][          0] )
    print( df_resultado[linha][ coluna + 3] )
    print( df_resultado[linha][ coluna + 4] )
    print()
    
    # calcula precisão para fornecedores
    for fornecedor in fornecedores:
        
        linha += 1
        
        # obtem dados do fornecedor
        mask = df["Remetente"] == fornecedor
        dados_fornecedor = df[ mask ]
   

        # Cria um Pipeline - Classificador Composto
        # vectorizer => transformer => classifier 
        text_clf = Pipeline([('counts' , CountVectorizer() ),
                           #  ('tfidf', TfidfTransformer()),
                             ('classifier'  , MultinomialNB()   ),
                            ])

        # registra resultados
        df_resultado[linha][          0] = fornecedor
        df_resultado[linha][ coluna + 1] = dados_fornecedor.shape[0]
        df_resultado[linha][ coluna + 2] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], 
                                                                   dados_fornecedor['Categoria'])
        print(linha,' : ',coluna+1)
        print( df_resultado[linha][          0] )
        print( df_resultado[linha][ coluna + 1] )
        print( df_resultado[linha][ coluna + 2] )
        print()
    
        # Cria um Pipeline - Classificador Composto
        # vectorizer => transformer => classifier 
        text_clf = Pipeline([('counts' , CountVectorizer() ),
                             ('tfidf', TfidfTransformer()),
                             ('classifier'  , MultinomialNB()   ),
                            ])

        # registra resultados
        df_resultado[linha][ coluna + 3] = dados_fornecedor.shape[0]
        df_resultado[linha][ coluna + 4] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], 
                                                                   dados_fornecedor['Categoria'])         
        print(linha,' : ',coluna+3)
        print( df_resultado[linha][          0] )
        print( df_resultado[linha][ coluna + 3] )
        print( df_resultado[linha][ coluna + 4] )
        print()
    
    
    

In [ ]:
df_resultado

In [ ]:
# create the transform
vectorizer = CountVectorizer()

In [ ]:
text = df['NovaDescricao']

In [ ]:
# tokenize and build vocab
vectorizer.fit(text)

In [ ]:
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
palavras = list(vectorizer.vocabulary_)

In [ ]:
palavras.sort()

In [ ]:
len(palavras)

In [ ]:
palavras

In [ ]:
df_palavras = pd.DataFrame(data = palavras)


In [ ]:
df_palavras.to_csv('palavras.cvs', sep=";", index=False )

In [ ]:
pwd

In [ ]:
vocabulário = list( vectorizer.vocabulary_ )
vocabulário

In [ ]:
vocabulário.sort()
vocabulário

# Mineração